In [20]:
import pandas as pd

Website for downloading data: https://data.marine.copernicus.eu/product/INSITU_BAL_PHYBGCWAV_DISCRETE_MYNRT_013_032/download?dataset=cmems_obs-ins_bal_phybgcwav_mynrt_na_irr_202311--ext--history

In [21]:
times = pd.date_range(start="2014-01-01", end="2023-12-31", freq='30min')

In [25]:
def prepare_dfs(df):
    # Since it is timezone UTC, we can safely remove the 'Z' and 'T' characters and convert to datetime
    df["datetime"]=pd.to_datetime(df['time'].str.replace('T', ' ').str.replace('Z', ''))
    # Only choose every 30 minute data. It is different how frequent the data is, hence the funny looking indexing
    df_30min = df[df['datetime'].isin(times)]
    # Select only data with valueQc < 2 (good or probably good data)
    df_good = df_30min[df_30min['valueQc'] < 2]
    # Select only the columns we want to keep
    df_new = df_good.set_index('datetime')[['platformId', 'value']]
    # rename 'value' to 'SLEV' 
    df_new.rename(columns={'value': 'water_level'}, inplace=True)
    # Round values
    df_new['water_level'] = df_new['water_level'].round(4)
    # Now split df_new into separate dataframes for each platformId and collect them in a lookup dictionary
    platforms = df_new['platformId'].unique()
    dfs = {}
    for platform in platforms:
        dfs[platform] = df_new[df_new['platformId'] == platform]

    return dfs


In [26]:
files = ["cmems_obs_2014_2015.csv", "cmems_obs_2016_2017.csv", "cmems_obs_2018_2019.csv", "cmems_obs_2020_2021.csv", "cmems_obs_2022_2023.csv"]

df =pd.read_csv(f"raw_data/{files[0]}", index_col=0, parse_dates=True)
dfs_all = prepare_dfs(df)

# Loop over files (from old to new) and add to dfs_all
for f in files[1:]:
    df =pd.read_csv(f"raw_data/{f}", index_col=0, parse_dates=True)
    dfs_next = prepare_dfs(df)

    # Concatenate the dataframes in dfs0 and dfs1 according to their keys, avoiding duplicated rows
    for key in dfs_all.keys():
        if key in dfs_next.keys():
            dfs_next[key] = dfs_next[key].loc[dfs_next[key].index > dfs_all[key].index[-1]]
            # Concatenate new data to existing data
            dfs_all[key] = pd.concat([dfs_all[key], dfs_next[key]])

    # If new stations are present, add them to the dictionary
    for key in dfs_next.keys():
        if key not in dfs_all.keys():
            dfs_all[key] = dfs_next[key]


In [27]:
# Check dfs_all for duplicate indices
for key in dfs_all.keys():
    if dfs_all[key].index.duplicated().any():
        print(f"Duplicate indices in {key}")

dfs_all.keys()

dict_keys(['Drogden', 'Klagshamn', 'Koege', 'Dragor', 'Skanor', 'Hornbaek', 'Kobenhavn', 'Viken', 'Vedbaek', 'Barseback', 'NordreRose', 'Flinten7', 'MalmoHamn', 'Helsingborg'])

In [28]:
# Go through the keys and save the dataframes to csv
# Add to the name the first present year in data and the last present year
skip_station = ["Viken"] # Skip this station since it is too far out of the domain
for key in dfs_all.keys():
    if key in skip_station:
        continue
    dfs_all[key].drop(columns=["platformId"]).to_csv(f"../observations/{key}_wl.csv", date_format='%Y-%m-%dT%H:%M:%SZ')